# Science User Case - Inspecting a Candidate List

In [184]:
import warnings
warnings.filterwarnings("ignore")
import sys
sys.path.append('workshop-dev-master')
from navo_utils.registry import Registry
from navo_utils.image import Image, ImageColumn
from navo_utils.spectra import Spectra, SpectraColumn
from navo_utils.cone import Cone
from navo_utils.tap import Tap
from navo_utils.utils import astropy_table_from_votable_response
from astropy.table import Table

# Utility functions used in the workshop.
import navo_utils.utils

Find available data for a list of objects. Here's the paper: https://ui.adsabs.harvard.edu//#abs/2016ApJ...817..109O/abstract

Table 1 has a list of super spirals.

## Search NED for objects in this refcode

In [185]:
from astroquery.ned import Ned
objects_in_paper = Ned.query_refcode('2016ApJ...817..109O')
objects_in_paper.show_in_notebook()

idx,No.,Object Name,RA(deg),DEC(deg),Type,Velocity,Redshift,Redshift Flag,Magnitude and Filter,Distance (arcmin),References,Notes,Photometry Points,Positions,Redshift Points,Diameter Points,Associations
,,,degrees,degrees,,km / s,,,,arcm,,,,,,,
0,1,2MASX J00155012-1002427,3.95892,-10.045109999999999,G,52788.0,0.17608299999999999,,17.5g,--,10,0,51,5,3,10,0
1,2,2MASX J00380781-0109365,9.5325199999999999,-1.1602399999999999,G,62565.0,0.20869499999999999,,18.0g,--,10,0,59,7,3,10,0
2,3,2MASS J04042287-0541349,61.095480000000002,-5.6930300000000003,G,75138.0,0.250635,,18.6g,--,4,0,49,7,4,8,0
3,4,2MASX J07380615+2823592,114.52567999999999,28.39995,G,69255.0,0.23100999999999999,,18.0g,--,8,0,54,5,3,10,0
4,5,2MASX J07550424+1353261,118.76776,13.890650000000001,G,66754.0,0.22266900000000001,,18.5g,--,7,0,38,5,3,6,0
5,6,2MASX J08265512+1811476,126.72969000000001,18.196650000000002,G,79608.0,0.26554299999999997,,18.4g,--,4,0,34,4,3,6,0
6,7,2MASX J08542169+0449308,133.59032999999999,4.8251099999999996,G,47018.0,0.156835,,16.7g,--,12,0,58,6,3,10,0
7,8,GMBCG J137.43670+22.43538,137.4367,22.435390000000002,GClstr,90837.0,0.30299999999999999,PHOT,,--,2,0,0,0,1,0,0
8,9,2MASX J09094480+2226078,137.4367,22.435379999999999,G,85557.0,0.28538599999999997,,19.0g,--,5,0,30,3,4,6,0


## Remove the non-galaxies from the table.

In [186]:
#Let's see whether we are dealing with strings or byte strings
objects_in_paper['Type'].data[0]

b'G'

In [187]:
#Byte strings it is!
galaxies = objects_in_paper[objects_in_paper['Type'] == b'G']

galaxies.show_in_notebook()

idx,No.,Object Name,RA(deg),DEC(deg),Type,Velocity,Redshift,Redshift Flag,Magnitude and Filter,Distance (arcmin),References,Notes,Photometry Points,Positions,Redshift Points,Diameter Points,Associations
,,,degrees,degrees,,km / s,,,,arcm,,,,,,,
0,1,2MASX J00155012-1002427,3.95892,-10.045109999999999,G,52788.0,0.17608299999999999,,17.5g,--,10,0,51,5,3,10,0
1,2,2MASX J00380781-0109365,9.5325199999999999,-1.1602399999999999,G,62565.0,0.20869499999999999,,18.0g,--,10,0,59,7,3,10,0
2,3,2MASS J04042287-0541349,61.095480000000002,-5.6930300000000003,G,75138.0,0.250635,,18.6g,--,4,0,49,7,4,8,0
3,4,2MASX J07380615+2823592,114.52567999999999,28.39995,G,69255.0,0.23100999999999999,,18.0g,--,8,0,54,5,3,10,0
4,5,2MASX J07550424+1353261,118.76776,13.890650000000001,G,66754.0,0.22266900000000001,,18.5g,--,7,0,38,5,3,6,0
5,6,2MASX J08265512+1811476,126.72969000000001,18.196650000000002,G,79608.0,0.26554299999999997,,18.4g,--,4,0,34,4,3,6,0
6,7,2MASX J08542169+0449308,133.59032999999999,4.8251099999999996,G,47018.0,0.156835,,16.7g,--,12,0,58,6,3,10,0
7,9,2MASX J09094480+2226078,137.4367,22.435379999999999,G,85557.0,0.28538599999999997,,19.0g,--,5,0,30,3,4,6,0
8,10,2MASX J09260805+2405242,141.53357,24.090039999999998,G,66689.0,0.22245100000000001,,17.8g,--,10,0,32,4,5,6,0


## Grab the SDSS images for each super spiral

In [ ]:
from astroquery.sdss import SDSS
from astropy.coordinates import SkyCoord  # High-level coordinates
from astropy.coordinates import ICRS, Galactic, FK4, FK5  # Low-level frames
from astropy.coordinates import Angle, Latitude, Longitude  # Angles
import astropy.units as u

for object in galaxies:
    pos = SkyCoord(object['RA(deg)'], object['DEC(deg)'], frame="icrs", unit=(u.deg, u.deg))
    xid = SDSS.query_region(pos, spectro=True) #requires matches to have spectroscopy, not just photometry:
    im = SDSS.get_images(matches=xid, band='g')

## Grab the WISE images for each super spiral

### Search the Registry for WISE image services.

In [ ]:
wise_services = Registry.query(keyword='allwise', service_type='image')

print(f'{len(wise_services)} result(s) found.')

wise_services

In [ ]:
wise_service = wise_services[0]['access_url']
wise_service

### Query the AllWISE image service for those that cover the Super Spirals.

In [ ]:
from navo_utils.image import Image, ImageColumn

size = '0'  #diameter in degrees
ra = objects_in_paper[0]['RA(deg)']
dec = objects_in_paper[0]['DEC(deg)']

# Perform the query on the WISE service.
pos = SkyCoord(ra, dec, frame="icrs", unit=u.deg)
print(pos)
print(wise_services[0]['access_url'])
result_list = Image.query(coords=pos, radius=size, service=wise_services[0])

# The result is a list of tables, one for each coordinate in the query.
# In this case, we only input one coordinate, so there will only be one result 
# table in the list.
table = result_list[0]
table.show_in_notebook()

In [ ]:
missions = ['GALEX', '2MASS', 'AllWISE']

base_urls = ['http://mast.stsci.edu/portal_vo/Mashup/VoQuery.asmx/SiaV1?MISSION=GALEX&amp;', 
             'http://irsa.ipac.caltech.edu/ibe/sia/twomass/allsky/allsky?',
             'https://irsa.ipac.caltech.edu/ibe/sia/wise/allwise/p3am_cdd?']

#Create an astropy table with this info.
search_list = Table([missions, base_urls], names = ('mission', 'access_url'))

#Show this table.
#search_list.show_in_notebook()

sc = SkyCoord(185.47873, 4.47365, unit=u.deg)

# Choose a search size. Zero means the image must contain the search point.
size = '0'  #diameter in degrees

# Perform the query on the 2MASS service.
print(sc)
print(size)
print(search_list[2])
result_list = Image.query(coords=sc, radius=size, service=search_list[2])
print(result_list)
# The result is a list of tables, one for each coordinate in the query.
# In this case, we only input one coordinate, so there will only be one result 
# table in the list.
#table = result_list[0]
#table.show_in_notebook()


In [ ]:
import workshop_utils.utils
url = 'https://irsa.ipac.caltech.edu/ibe/data/wise/allwise/p3am_cdd/18/1853/1853p045_ac51/1853p045_ac51-w1-int-3.fits'
out_file_path = workshop_utils.utils.download_file(url, directory='.')

## Grab the HST images for each super spiral

## Plot the SDSS and WISE images next to each other